# 멜론 크롤링
1.노래 리스트 받아와서 쿼리 바꿔서 검색
그 안에서 id 찾아서 -> 노래 id 리스트업
2.노래 id로 노래상세페이지 쿼리바꿔가면서 데이터 크롤링 - 발매일, 장르, 좋아요수, 댓글수

In [9]:
import time
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests

In [4]:
# 1. 노래 리스트에서 가져와서 노래별로 id 찾기
music_list = ['and july', 'cupid', '꽃']
singer_list = ['heize', '피프티피프티', '지수']

# 띄어쓰기는 +로 바꿔주기
music_list = [m.replace(' ', '+') for m in music_list]
singer_list = [s.replace(' ', '+') for s in singer_list]

In [135]:

# 2. 노래 id 찾아서 code_list 만들기
# https://www.melon.com/search/total/index.htm?q=heize+and+july

code_list=[]
for idx, music in enumerate(music_list):
    base_url = 'https://www.melon.com/search/total/index.htm?q='
    URL = base_url + singer_list[idx] + '+' + music
    header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
    response = requests.get(URL,headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
    code = links.attrs['value']
    code_list.append(code)
code_list

['8271355', '36206259', '36331121']

In [145]:
# 노래 id로 노래상세페이지 쿼리바꿔가면서 데이터 크롤링 - 발매일, 장르, 좋아요수, 댓글수
# 뮤비 데이터 - 좋아요, 뷰

import json
import re

for code in code_list:
    header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
    URL = f'https://www.melon.com/song/detail.htm?songId={code}'
    response_1 = requests.get(URL,headers=header)
    soup = BeautifulSoup(response_1.text, 'html.parser')

    # 발매일, 장르
    release_date = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)').text
    category = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)').text
    
    # 뮤비 고유 code 가져오기
    movie = soup.select_one('#conts > div.section_movie > div.service_list_video.type03.d_video_list > ul > li > div.thumb > a')
    movie = movie.attrs['href']
    movie = re.sub('[a-zA-z:.(\')]', '', movie)
    movie_code = movie.split(', ')[1]

    # 뮤비 고유 코드 이용해서 json 접근해서 뮤비 뷰수와 좋아요 가져오기
    URL_3 = f'https://www.melon.com/commonlike/getMvLikeWithReadCnt.json?contsIds={movie_code}'
    response_3 = requests.get(URL_3, headers=header).text
    movie_json = json.loads(response_3)
    movie_like = movie_json['contsLike'][0]['LOVECNT']
    movie_views = movie_json['contsLike'][0]['READCNT']
    print(f'뮤비 코드:{movie_code} 뮤비좋아요:{movie_like}, 뮤비뷰수{movie_views}')

    # 좋아요 가져오기위해 json으로 접근
    URL_2 = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={code}'
    response_2 = requests.get(URL_2,headers=header).text

    like_json = json.loads(response_2)
    like_count = like_json['contsLike'][0]['SUMMCNT']

    print(f'노래코드:{code} 발매일{release_date}, 장르{category} 좋아요{like_count}')
    print('--------------------------------------------------------------')

뮤비 코드:50152059 뮤비좋아요:2309, 뮤비뷰수588781
노래코드:8271355 발매일2016.07.18, 장르랩/힙합 좋아요182074
--------------------------------------------------------------
뮤비 코드:50260014 뮤비좋아요:217, 뮤비뷰수8829
노래코드:36206259 발매일2023.02.24, 장르댄스 좋아요50755
--------------------------------------------------------------
뮤비 코드:50260334 뮤비좋아요:1488, 뮤비뷰수116284
노래코드:36331121 발매일2023.03.31, 장르댄스 좋아요86179
--------------------------------------------------------------
